# SET UP

In [1]:
import pandas as pd
import os
import tensorflow as tf
import torch
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout
from keras.optimizers import Adam
import random

In [2]:
print(tf.__version__)

2.14.0


In [4]:
! pip show keras

Name: keras
Version: 2.14.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow


In [ ]:
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/Data255/"
directory = "/content/drive/MyDrive/Data255/"

Mounted at /content/drive
'Bản sao của Project - DNN.ipynb'   Doc2Vec.model		   HW4
 BertEmbeddings.ipynb		    Doc2Vec.model.syn1neg.npy	   Other
 CleanedData.csv		    Doc2Vec.model.wv.vectors.npy  'Project - DNN.ipynb'
 CleanedData.gsheet		    glove.6B.100d.txt		   wiki-news-300d-1M.vec
 Doc2vec			    glove.6B.50d.txt		   Word2vec
 doc2vec.csv			    HW3				   word2vec.csv


In [ ]:
raw_data = pd.read_csv(directory + 'CleanedData.csv')
df = pd.DataFrame(raw_data)

In [ ]:
df

,text,target
0,ankara reuters turkish sudanese intelligence a...,0
1,reuters supreme court friday refused stay ruli...,0
2,one presidential candidate king dog whistle po...,1
3,wonder want anyone see speech singing praise t...,1
4,feel bern lot gullible young people fed big li...,1
...,...,...
42993,washington reuters democratic presidential nom...,0
42994,vatican city washington reuters democratic pre...,0
42995,last week new york giant player odell beckham ...,1
42996,almaty reuters kazakhstan change official alph...,0


In [ ]:
df['text'] = df['text'].astype(str)

In [ ]:
#add length to the end
#df['length_info'] = df['text'].apply(lambda x: f"{x} {len(x)}")

#add length to the beginning
df['length_info'] = df['text'].apply(lambda x: f'{len(x)}')

In [ ]:
df['length_info'] = df['length_info'].astype(float)
#df['length_info'] = pd.to_numeric(df['length_info'], errors='coerce')

In [ ]:
df

,text,target,length_info
0,ankara reuters turkish sudanese intelligence a...,0,1028.0
1,reuters supreme court friday refused stay ruli...,0,1145.0
2,one presidential candidate king dog whistle po...,1,1582.0
3,wonder want anyone see speech singing praise t...,1,633.0
4,feel bern lot gullible young people fed big li...,1,1090.0
...,...,...,...
42993,washington reuters democratic presidential nom...,0,4126.0
42994,vatican city washington reuters democratic pre...,0,680.0
42995,last week new york giant player odell beckham ...,1,1030.0
42996,almaty reuters kazakhstan change official alph...,0,1402.0


In [ ]:
#Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(df[['text','length_info']], df['target'], test_size=0.3, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state = 42)

In [ ]:
X_train

,text,length_info
22049,oops mistake mistake one time understandable d...,1151.0
36793,ankara reuters president tayyip erdogan dismis...,1047.0
4220,scared death prospect donald trump president u...,2192.0
2545,thank alabama protecting history good bad impo...,1063.0
1482,whiskey tango hotel worry brooke clarified lud...,2285.0
...,...,...
6265,sochi russia reuters critical decision taken s...,341.0
11284,beijing reuters china defended ally pakistan t...,1465.0
38158,sydney reuters australian prime minister malco...,777.0
860,sean spicer former communication man republica...,2747.0


In [ ]:
y_train

22049    1
36793    0
4220     1
2545     1
1482     1
        ..
6265     0
11284    0
38158    0
860      1
15795    1
Name: target, Length: 30098, dtype: int64

# MODEL FUNCTION

In [ ]:
def create_dnn_model(data):
  model = tf.keras.Sequential()
  model.add(Dense(128, input_dim=data.shape[1], activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
  return model

# 1. Doc2Vec embeddings training - No need to run

In [ ]:
# Approach: Get Doc2Vec embeddings for each document

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Tokenize and tag the documents for Doc2Vec
documents = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(df['text'])]

In [ ]:
# Train a Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=20)
doc2vec_model.build_vocab(documents)
doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [ ]:
doc2vec_model.save(directory + "Doc2Vec.model")

## Doc2Vec embeddings load pretrained model - No need to run

In [ ]:
from gensim.models import Doc2Vec
doc2vec_model = Doc2Vec.load(directory + "Doc2Vec.model")

In [ ]:
def get_doc2vec_embeddings(text):
    words = text.split()
    vector = doc2vec_model.infer_vector(words)
    return vector

In [ ]:
#df['doc2vec_embeddings'] = df['text'].apply(get_doc2vec_embeddings)
doc2vec_X_train = X_train['text'].apply(get_doc2vec_embeddings)
doc2vec_X_test = X_test['text'].apply(get_doc2vec_embeddings)
doc2vec_X_val = X_val['text'].apply(get_doc2vec_embeddings)

In [ ]:
doc2vec_X_train.to_csv(directory + "Doc2vec/doc2vec_X_train_withlen.csv", index=False)
doc2vec_X_test.to_csv(directory + "Doc2vec/doc2vec_X_test_withlen.csv", index=False)
doc2vec_X_val.to_csv(directory + "Doc2vec/doc2vec_X_val_withlen.csv", index=False)

# Doc2Vec - Load file

In [ ]:
doc2vec_train = pd.read_csv(directory + "Doc2vec/doc2vec_X_train_withlen.csv")
doc2vec_test = pd.read_csv(directory + "Doc2vec/doc2vec_X_test_withlen.csv")
doc2vec_val = pd.read_csv(directory + "Doc2vec/doc2vec_X_val_withlen.csv")

doc2vec_X_train_load = pd.DataFrame(doc2vec_train)
doc2vec_X_test_load = pd.DataFrame(doc2vec_test)
doc2vec_X_val_load = pd.DataFrame(doc2vec_val)

In [ ]:
doc2vec_X_train_load

,text
0,[-9.24201965e-01 -3.94912690e-01 -6.11960180e-...
1,[ 4.05917466e-01 2.86634356e-01 7.33916759e-...
2,[-4.85245734e-01 3.62445235e-01 -8.94320369e-...
3,[ 0.6553484 1.1728262 0.56908494 -0.676199...
4,[-7.35291123e-01 -9.51013088e-01 1.18234217e+...
...,...
30093,[ 4.96687323e-01 1.17610030e-01 -1.68740928e-...
30094,[-0.33620912 0.4459364 -0.41177645 -0.687118...
30095,[ 0.02652752 0.05386706 -0.03928535 0.195386...
30096,[ 1.38650760e-01 3.02808821e-01 1.10059142e+...


In [ ]:
doc2vec_X_train_load = doc2vec_X_train_load['text'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
doc2vec_X_test_load = doc2vec_X_test_load['text'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
doc2vec_X_val_load = doc2vec_X_val_load['text'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

In [ ]:
doc2vec_X_train_load = doc2vec_X_train_load.apply(lambda x: [float(i) for i in x])
doc2vec_X_test_load = doc2vec_X_test_load.apply(lambda x: [float(i) for i in x])
doc2vec_X_val_load = doc2vec_X_val_load.apply(lambda x: [float(i) for i in x])

In [ ]:
type(doc2vec_X_train_load.iloc[0][0])

float

In [ ]:
doc2vec_X_train_load.shape

(30098,)

In [ ]:
min_values_doc2vec = min(doc2vec_X_train_load.apply(lambda x: min(x)))
max_values_doc2vec = max(doc2vec_X_train_load.apply(lambda x: max(x)))

In [ ]:
# Normalize to the range (a, b)
normalized_data_X_train_doc2vec = min_values_doc2vec + (X_train['length_info'] - X_train['length_info'].min()) * (max_values_doc2vec - min_values_doc2vec) / (X_train['length_info'].max() - X_train['length_info'].min())
normalized_data_X_test_doc2vec = min_values_doc2vec + (X_test['length_info'] - X_test['length_info'].min()) * (max_values_doc2vec - min_values_doc2vec) / (X_test['length_info'].max() - X_test['length_info'].min())
normalized_data_X_val_doc2vec = min_values_doc2vec + (X_val['length_info'] - X_val['length_info'].min()) * (max_values_doc2vec - min_values_doc2vec) / (X_val['length_info'].max() - X_val['length_info'].min())

In [ ]:
doc2vec_X_train_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(doc2vec_X_train_load, normalized_data_X_train_doc2vec)])
doc2vec_X_test_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(doc2vec_X_test_load, normalized_data_X_test_doc2vec)])
doc2vec_X_val_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(doc2vec_X_val_load, normalized_data_X_val_doc2vec)])

In [ ]:
doc2vec_X_train = np.array(doc2vec_X_train_load.tolist())
doc2vec_X_test = np.array(doc2vec_X_test_load.tolist())
doc2vec_X_val = np.array(doc2vec_X_val_load.tolist())

doc2vec_X_train_withlen = np.array(doc2vec_X_train_withlen.tolist())
doc2vec_X_test_withlen = np.array(doc2vec_X_test_withlen.tolist())
doc2vec_X_val_withlen = np.array(doc2vec_X_val_withlen.tolist())

In [ ]:
doc2vec_X_train

array([[-0.92420196, -0.39491269, -0.06119602, ..., -1.21163249,
         0.41028154, -0.11778655],
       [ 0.40591747,  0.28663436,  0.07339168, ...,  0.38373429,
         0.65907335,  0.85271257],
       [-0.48524573,  0.36244523, -0.89432037, ..., -0.03247582,
         0.85231566, -0.20682625],
       ...,
       [ 0.02652752,  0.05386706, -0.03928535, ..., -0.00751448,
        -0.08940426,  0.01674267],
       [ 0.13865076,  0.30280882,  1.10059142, ...,  0.01373419,
        -0.16091536,  0.08778293],
       [ 0.41091472,  1.0942167 , -0.87838149, ...,  1.32015073,
        -0.7358095 ,  0.57479232]])

In [ ]:
doc2vec_X_train_withlen.shape

(30098, 301)

# Doc2Vec - model without length

In [ ]:
model_doc2vec = create_dnn_model(doc2vec_X_train)
model_doc2vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_doc2vec.fit(doc2vec_X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 10s 3ms/step - loss: 0.2728 - accuracy: 0.8894
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1887 - accuracy: 0.9296
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1650 - accuracy: 0.9406
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1498 - accuracy: 0.9474
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1369 - accuracy: 0.9511
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1226 - accuracy: 0.9569
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1101 - accuracy: 0.9609
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1001 - accuracy: 0.9640
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0931 - accuracy: 0.9669
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0889 - accuracy: 0.967

In [ ]:
# Evaluate the model
loss, accuracy = model_doc2vec.evaluate(doc2vec_X_val, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1733 - accuracy: 0.9460
Test loss: 0.17332884669303894
Test accuracy: 0.9460465312004089


# Doc2Vec - model with length

In [ ]:
model_doc2vec_withlen = create_dnn_model(doc2vec_X_train_withlen)
model_doc2vec_withlen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_doc2vec_withlen.fit(doc2vec_X_train_withlen, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 4s 3ms/step - loss: 0.2768 - accuracy: 0.8867
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1874 - accuracy: 0.9318
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1611 - accuracy: 0.9416
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1420 - accuracy: 0.9493
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1309 - accuracy: 0.9532
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1197 - accuracy: 0.9556
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1125 - accuracy: 0.9588
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1013 - accuracy: 0.9643
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0943 - accuracy: 0.9657
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0888 - accuracy: 0.9685

In [ ]:
# Evaluate the model
loss, accuracy = model_doc2vec_withlen.evaluate(doc2vec_X_val_withlen, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1671 - accuracy: 0.9471
Test loss: 0.16712254285812378
Test accuracy: 0.9471318125724792


# Doc2Vec - Active Learning with length

In [ ]:
# Active learning loop
query_indices = random.sample(range(len(doc2vec_X_train_withlen)), k=100)  # Initialize with a random set of instances

In [ ]:
# Set the number of active learning iterations
num_iterations = 10

for i in range(num_iterations):
    model_doc2vec_withlen_active = create_dnn_model(doc2vec_X_train_withlen)
    model_doc2vec_withlen_active.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model on the current training set
    model_doc2vec_withlen_active.fit(doc2vec_X_train_withlen[query_indices], y_train.iloc[query_indices], epochs=10, batch_size=32, verbose=1)

    # Predict on the test set
    test_predictions = (model_doc2vec_withlen_active.predict(doc2vec_X_val_withlen) > 0.5).astype(int)

    # Evaluate the model
    accuracy = accuracy_score(y_val, test_predictions)
    print(f"Iteration {i + 1} - Validation Accuracy: {accuracy}")

    # Query the instances with the least confidence
    uncertainty_scores = np.abs(model_doc2vec_withlen_active.predict(doc2vec_X_train_withlen) - 0.5).flatten()
    #query_indices = np.argsort(uncertainty_scores)[-100:]

    # Add the queried instances to the training set
    query_indices = np.concatenate((query_indices, np.argsort(uncertainty_scores)[-100:]))


Epoch 1/10
4/4 [==============================] - 1s 6ms/step - loss: 0.7761 - accuracy: 0.4700
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 0.7036 - accuracy: 0.6200
Epoch 3/10
4/4 [==============================] - 0s 5ms/step - loss: 0.6435 - accuracy: 0.6200
Epoch 4/10
4/4 [==============================] - 0s 5ms/step - loss: 0.5760 - accuracy: 0.7000
Epoch 5/10
4/4 [==============================] - 0s 5ms/step - loss: 0.5542 - accuracy: 0.7100
Epoch 6/10
4/4 [==============================] - 0s 5ms/step - loss: 0.5409 - accuracy: 0.7400
Epoch 7/10
4/4 [==============================] - 0s 5ms/step - loss: 0.4946 - accuracy: 0.7700
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4474 - accuracy: 0.7800
Epoch 9/10
4/4 [==============================] - 0s 4ms/step - loss: 0.5149 - accuracy: 0.7200
Epoch 10/10
202/202 [==============================] - 0s 2ms/step
Iteration 1 - Validation Accuracy: 0.7494573643410852
941/941 [======

In [ ]:
# Final evaluation on the test set
final_test_predictions = (model_doc2vec_withlen_active.predict(doc2vec_X_val_withlen) > 0.5).astype(int)
final_accuracy = accuracy_score(y_val, final_test_predictions)
print(f"Final Validation Accuracy: {final_accuracy}")


202/202 [==============================] - 0s 2ms/step
Final Validation Accuracy: 0.872093023255814


# 2. GloVe embeddings

In [ ]:
from gensim.models import KeyedVectors
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer

In [ ]:
embeddings_index = dict()
f = open(directory + 'glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
# Tokenize the text
tokenizer = Tokenizer()
#tokenizer.fit_on_texts(X_train['text'])
tokenizer.fit_on_texts(df['text'])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train['text'])
X_test_seq = tokenizer.texts_to_sequences(X_test['text'])
X_val_seq = tokenizer.texts_to_sequences(X_val['text'])

# Pad sequences to have the same length
max_sequence_length = 512
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_sequence_length)

In [ ]:
# Initialize an embedding matrix
embedding_dim = 50  # You're using the 50-dimensional GloVe embeddings

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45769   ,  0.85191   ,  0.31097999, ..., -0.74225998,
         0.77052999,  0.26394001],
       [ 0.38973001, -0.2121    ,  0.51836997, ...,  0.86119002,
         0.1415    ,  1.20179999],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

#Glove Model without length

In [ ]:
model = Sequential()
#model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=X_train_pad.shape[1], weights=[embedding_matrix], trainable=False))
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim,input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Flatten())
model.add(Dense(128, input_dim=X_train_pad.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 7s 4ms/step - loss: 0.3210 - accuracy: 0.8702
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2078 - accuracy: 0.9205
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1666 - accuracy: 0.9384
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1307 - accuracy: 0.9504
Epoch 5/10
941/941 [==============================] - 3s 4ms/step - loss: 0.1115 - accuracy: 0.9581
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0950 - accuracy: 0.9624
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0849 - accuracy: 0.9666
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0789 - accuracy: 0.9690
Epoch 9/10
941/941 [==============================] - 3s 4ms/step - loss: 0.0718 - accuracy: 0.9703
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0665 - accuracy: 0.9723

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val_pad, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.2558 - accuracy: 0.9274
Test loss: 0.25575658679008484
Test accuracy: 0.9274418354034424


# Glove Model with length

In [ ]:
X_train_pad.shape

(30098, 512)

In [ ]:
glove_X_train = pd.Series(X_train_pad.tolist())
glove_X_test = pd.Series(X_test_pad.tolist())
glove_X_val = pd.Series(X_val_pad.tolist())

In [ ]:
min_values_glove = min(glove_X_train.apply(lambda x: min(x)))
max_values_glove = max(glove_X_train.apply(lambda x: max(x)))

In [ ]:
# Normalize to the range (a, b)
normalized_data_X_train_glove = min_values_glove + (X_train['length_info'] - X_train['length_info'].min()) * (max_values_glove - min_values_glove) / (X_train['length_info'].max() - X_train['length_info'].min())
normalized_data_X_test_glove = min_values_glove + (X_test['length_info'] - X_test['length_info'].min()) * (max_values_glove - min_values_glove) / (X_test['length_info'].max() - X_test['length_info'].min())
normalized_data_X_val_glove = min_values_glove + (X_val['length_info'] - X_val['length_info'].min()) * (max_values_glove - min_values_glove) / (X_val['length_info'].max() - X_val['length_info'].min())

In [ ]:
glove_X_train_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(glove_X_train, normalized_data_X_train_glove)])
glove_X_test_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(glove_X_test, normalized_data_X_test_glove)])
glove_X_val_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(glove_X_val, normalized_data_X_val_glove)])

In [ ]:
glove_X_train_withlen = np.array(glove_X_train_withlen.tolist())
glove_X_test_withlen = np.array(glove_X_test_withlen.tolist())
glove_X_val_withlen = np.array(glove_X_val_withlen.tolist())

In [ ]:
glove_X_train_withlen.shape

(30098, 513)

In [ ]:
glove_model = Sequential()
glove_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim,input_length=max_sequence_length+1, weights=[embedding_matrix], trainable=False))
glove_model.add(Flatten())
glove_model.add(Dense(128, input_dim=glove_X_train_withlen.shape[1], activation='relu'))
glove_model.add(Dropout(0.5))
glove_model.add(Dense(64, activation='relu'))
glove_model.add(Dropout(0.5))
glove_model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

glove_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
glove_model.fit(glove_X_train_withlen, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 4s 3ms/step - loss: 0.3124 - accuracy: 0.8739
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2018 - accuracy: 0.9226
Epoch 3/10
941/941 [==============================] - 3s 4ms/step - loss: 0.1567 - accuracy: 0.9393
Epoch 4/10
941/941 [==============================] - 3s 4ms/step - loss: 0.1296 - accuracy: 0.9496
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1038 - accuracy: 0.9597
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0939 - accuracy: 0.9633
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0821 - accuracy: 0.9677
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0732 - accuracy: 0.9699
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0686 - accuracy: 0.9717
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.0623 - accuracy: 0.9745

In [ ]:
# Evaluate the model
loss, accuracy = glove_model.evaluate(glove_X_val_withlen, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.2872 - accuracy: 0.9257
Test loss: 0.2872115671634674
Test accuracy: 0.9257364273071289


# GloVe - Active Learning

In [ ]:
max_sequence_length

512

In [ ]:
glove_X_train_withlen.shape[1]

513

In [ ]:
# Active learning loop
query_indices = random.sample(range(len(glove_X_train_withlen)), k=100)  # Initialize with a random set of instances

In [ ]:
# Set the number of active learning iterations
num_iterations = 10

for i in range(num_iterations):
    glove_model = Sequential()
    glove_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim,input_length=max_sequence_length + 1, weights=[embedding_matrix], trainable=False))
    glove_model.add(Flatten())
    glove_model.add(Dense(128, input_dim=glove_X_train_withlen.shape[1], activation='relu'))
    glove_model.add(Dropout(0.5))
    glove_model.add(Dense(64, activation='relu'))
    glove_model.add(Dropout(0.5))
    glove_model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

    glove_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model on the current training set
    glove_model.fit(glove_X_train_withlen[query_indices], y_train.iloc[query_indices], epochs=10, batch_size=32, verbose=1)

    # Predict on the test set
    test_predictions = (glove_model.predict(glove_X_val_withlen) > 0.5).astype(int)

    # Evaluate the model
    accuracy = accuracy_score(y_val, test_predictions)
    print(f"Iteration {i + 1} - Validation Accuracy: {accuracy}")

    # Query the instances with the least confidence
    uncertainty_scores = np.abs(glove_model.predict(glove_X_train_withlen) - 0.5).flatten()
    #query_indices = np.argsort(uncertainty_scores)[-100:]

    # Add the queried instances to the training set
    query_indices = np.concatenate((query_indices, np.argsort(uncertainty_scores)[-100:]))

Epoch 1/10
4/4 [==============================] - 1s 5ms/step - loss: 0.7347 - accuracy: 0.5500
Epoch 2/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6223 - accuracy: 0.7700
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6710 - accuracy: 0.8100
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 0.2399 - accuracy: 0.8900
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.2292 - accuracy: 0.9400
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 0.2821 - accuracy: 0.9100
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4654 - accuracy: 0.8900
Epoch 8/10
4/4 [==============================] - 0s 5ms/step - loss: 0.2298 - accuracy: 0.9600
Epoch 9/10
4/4 [==============================] - 0s 4ms/step - loss: 0.0606 - accuracy: 0.9800
Epoch 10/10
202/202 [==============================] - 0s 1ms/step
Iteration 1 - Validation Accuracy: 0.8114728682170542
941/941 [======

In [ ]:
# Final evaluation on the test set
final_test_predictions = (glove_model.predict(glove_X_val_withlen) > 0.5).astype(int)
final_accuracy = accuracy_score(y_val, final_test_predictions)
print(f"Final Validation Accuracy: {final_accuracy}")

202/202 [==============================] - 0s 1ms/step
Final Validation Accuracy: 0.8203100775193799


# 3. Word2Vec embeddings

In [ ]:
import gensim

In [ ]:
# Tokenize the text
df['tokens'] = df['text'].apply(lambda x: x.split())

# Train a Word2Vec model on your dataset
word2vec_model = gensim.models.Word2Vec(df['tokens'], vector_size=300, window=5, min_count=1, sg=0)

# Convert text to Word2Vec embeddings
def text_to_vector(text):
    words = text.split()
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return np.mean(vectors, axis=0)

In [ ]:
#df['word2vec_vectors'] = df['text'].apply(text_to_vector)

word2vec_X_train = X_train['text'].apply(text_to_vector)
word2vec_X_test = X_test['text'].apply(text_to_vector)
word2vec_X_val = X_val['text'].apply(text_to_vector)

In [ ]:
min_values = min(word2vec_X_train.apply(lambda x: min(x)))
max_values = max(word2vec_X_train.apply(lambda x: max(x)))

In [ ]:
# Normalize to the range (a, b)
normalized_data_X_train = min_values + (X_train['length_info'] - X_train['length_info'].min()) * (max_values - min_values) / (X_train['length_info'].max() - X_train['length_info'].min())
normalized_data_X_test = min_values + (X_test['length_info'] - X_test['length_info'].min()) * (max_values - min_values) / (X_test['length_info'].max() - X_test['length_info'].min())
normalized_data_X_val = min_values + (X_val['length_info'] - X_val['length_info'].min()) * (max_values - min_values) / (X_val['length_info'].max() - X_val['length_info'].min())

In [ ]:
word2vec_X_train_withlen = pd.Series([arr.tolist() + [length_info_item] for arr, length_info_item in zip(word2vec_X_train, normalized_data_X_train)])
word2vec_X_test_withlen = pd.Series([arr.tolist() + [length_info_item] for arr, length_info_item in zip(word2vec_X_test, normalized_data_X_test)])
word2vec_X_val_withlen = pd.Series([arr.tolist() + [length_info_item] for arr, length_info_item in zip(word2vec_X_val, normalized_data_X_val)])

In [ ]:
word2vec_X_train = np.array(word2vec_X_train.tolist())
word2vec_X_test = np.array(word2vec_X_test.tolist())
word2vec_X_val = np.array(word2vec_X_val.tolist())

word2vec_X_train_withlen = np.array(word2vec_X_train_withlen.tolist())
word2vec_X_test_withlen = np.array(word2vec_X_test_withlen.tolist())
word2vec_X_val_withlen = np.array(word2vec_X_val_withlen.tolist())

In [ ]:
word2vec_X_train

array([[-0.25454706, -0.05318117,  0.60905236, ..., -0.35340706,
        -0.01057071,  0.22687449],
       [-0.14937113, -0.12205982,  0.09835131, ..., -0.03408735,
        -0.11077782,  0.31822708],
       [-0.41626203, -0.01612945,  0.36325705, ..., -0.6065221 ,
        -0.05521131,  0.13753417],
       ...,
       [ 0.12650676, -0.08232406,  0.09658388, ..., -0.05594747,
         0.11651698,  0.28001866],
       [-0.17774756, -0.23495184,  0.4444936 , ..., -0.526826  ,
        -0.15983309,  0.2677571 ],
       [ 0.03409572,  0.08485754,  0.06129915, ..., -0.18756036,
        -0.09973911,  0.07471509]], dtype=float32)

# Word2Vec - Model without length

In [ ]:
model = create_dnn_model(word2vec_X_train)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(word2vec_X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 4s 3ms/step - loss: 0.1983 - accuracy: 0.9247
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1428 - accuracy: 0.9470
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1301 - accuracy: 0.9528
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1209 - accuracy: 0.9549
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1148 - accuracy: 0.9579
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1132 - accuracy: 0.9590
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1105 - accuracy: 0.9608
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1068 - accuracy: 0.9615
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1052 - accuracy: 0.9631
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1006 - accuracy: 0.9627

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(word2vec_X_val, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1080 - accuracy: 0.9600
Test loss: 0.1079879030585289
Test accuracy: 0.9599999785423279


# Word2Vec - Model with length

In [ ]:
model_word2vec_withlen = create_dnn_model(word2vec_X_train_withlen)
model_word2vec_withlen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word2vec_withlen.fit(word2vec_X_train_withlen, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 4s 3ms/step - loss: 0.1981 - accuracy: 0.9250
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1412 - accuracy: 0.9480
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1287 - accuracy: 0.9522
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1218 - accuracy: 0.9563
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1163 - accuracy: 0.9583
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1105 - accuracy: 0.9598
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1077 - accuracy: 0.9619
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1063 - accuracy: 0.9611
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1031 - accuracy: 0.9635
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1005 - accuracy: 0.9635

In [ ]:
# Evaluate the model
loss, accuracy = model_word2vec_withlen.evaluate(word2vec_X_val_withlen, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 0s 2ms/step - loss: 0.1047 - accuracy: 0.9648
Test loss: 0.10467497259378433
Test accuracy: 0.9648061990737915


### hyper parameter tuning learning rate = 1e-5

In [ ]:
model_word2vec_withlen = create_dnn_model(word2vec_X_train_withlen)
model_word2vec_withlen.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
#model_word2vec_withlen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word2vec_withlen.fit(word2vec_X_train_withlen, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 9s 3ms/step - loss: 0.6129 - accuracy: 0.6727
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.4588 - accuracy: 0.8185
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.3665 - accuracy: 0.8589
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.3163 - accuracy: 0.8804
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2895 - accuracy: 0.8882
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2729 - accuracy: 0.8959
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2583 - accuracy: 0.9010
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2452 - accuracy: 0.9074
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2406 - accuracy: 0.9109
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2296 - accuracy: 0.9132

In [ ]:
model_word2vec_withlen.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Evaluate the model
loss, accuracy = model_word2vec_withlen.evaluate(word2vec_X_val_withlen, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1960 - accuracy: 0.9250
Test loss: 0.1960461586713791
Test accuracy: 0.924961268901825


# Word2Vec - Active Learning with length

In [ ]:
#model_word2vec_withlen_active = create_dnn_model(word2vec_X_train_withlen)
#model_word2vec_withlen_active.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import random

In [ ]:
# Active learning loop
query_indices = random.sample(range(len(word2vec_X_train_withlen)), k=100)  # Initialize with a random set of instances

In [ ]:
# Set the number of active learning iterations
num_iterations = 10

for i in range(num_iterations):
    model_word2vec_withlen_active = create_dnn_model(word2vec_X_train_withlen)
    model_word2vec_withlen_active.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model on the current training set
    model_word2vec_withlen_active.fit(word2vec_X_train_withlen[query_indices], y_train.iloc[query_indices], epochs=10, batch_size=32, verbose=1)

    # Predict on the test set
    test_predictions = (model_word2vec_withlen_active.predict(word2vec_X_val_withlen) > 0.5).astype(int)

    # Evaluate the model
    accuracy = accuracy_score(y_val, test_predictions)
    print(f"Iteration {i + 1} - Validation Accuracy: {accuracy}")

    # Query the instances with the least confidence
    uncertainty_scores = np.abs(model_word2vec_withlen_active.predict(word2vec_X_train_withlen) - 0.5).flatten()
    #query_indices = np.argsort(uncertainty_scores)[-100:]

    # Add the queried instances to the training set
    query_indices = np.concatenate((query_indices, np.argsort(uncertainty_scores)[-100:]))


Epoch 1/10
4/4 [==============================] - 1s 5ms/step - loss: 0.7114 - accuracy: 0.4700
Epoch 2/10
4/4 [==============================] - 0s 4ms/step - loss: 0.5983 - accuracy: 0.7100
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 0.5363 - accuracy: 0.7700
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 0.5043 - accuracy: 0.7400
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4422 - accuracy: 0.8100
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 0.3898 - accuracy: 0.8100
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.3594 - accuracy: 0.8500
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.3226 - accuracy: 0.8500
Epoch 9/10
4/4 [==============================] - 0s 5ms/step - loss: 0.3490 - accuracy: 0.8500
Epoch 10/10
202/202 [==============================] - 0s 1ms/step
Iteration 1 - Validation Accuracy: 0.88
941/941 [====================

In [ ]:
len(query_indices)

1100

In [ ]:
# Final evaluation on the test set
final_test_predictions = (model_word2vec_withlen_active.predict(word2vec_X_val_withlen) > 0.5).astype(int)
final_accuracy = accuracy_score(y_val, final_test_predictions)
print(f"Final Validation Accuracy: {final_accuracy}")

202/202 [==============================] - 0s 1ms/step
Final Validation Accuracy: 0.8973643410852713


# Word2Vec - Active Learning Cross Validation

In [ ]:
from sklearn.model_selection import KFold

y_train_array = np.array(y_train.iloc[query_indices])

# Specify the number of folds for cross-validation
num_folds = 5

# Create a k-fold cross-validation object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize an array to store accuracy scores
cross_val_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(word2vec_X_train_withlen[query_indices]):
    X_train_fold, X_test_fold = word2vec_X_train_withlen[query_indices][train_index], word2vec_X_train_withlen[query_indices][test_index]
    y_train_fold, y_test_fold = y_train_array[train_index], y_train_array[test_index]

    # Create and compile the model
    model = create_dnn_model(X_train_fold)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32)

    # Evaluate the model on the test fold
    y_pred = (model.predict(X_test_fold) > 0.5).astype(int)
    accuracy = accuracy_score(y_test_fold, y_pred)

    # Store the accuracy score
    cross_val_scores.append(accuracy)

# Print the accuracy scores for each fold
for fold, accuracy in enumerate(cross_val_scores, start=1):
    print(f'Fold {fold}: Accuracy = {accuracy:.4f}')

# Print the mean and standard deviation of the accuracy scores
print(f'\nMean Accuracy: {np.mean(cross_val_scores):.4f}')
print(f'Standard Deviation: {np.std(cross_val_scores):.4f}')

Epoch 1/10
28/28 [==============================] - 1s 3ms/step - loss: 0.1532 - accuracy: 0.9341
Epoch 2/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0414 - accuracy: 0.9795
Epoch 3/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0295 - accuracy: 0.9864
Epoch 4/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0240 - accuracy: 0.9875
Epoch 5/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0188 - accuracy: 0.9932
Epoch 6/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0122 - accuracy: 0.9977
Epoch 7/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 0.9932
Epoch 8/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.9943
Epoch 9/10
28/28 [==============================] - 0s 3ms/step - loss: 0.0099 - accuracy: 0.9977
Epoch 10/10
7/7 [==============================] - 0s 2ms/step
Epoch 1/10
28/28 [==============================] - 1s 

# 4. BERT embeddings with attention mask

In [ ]:
pip install transformers

In [ ]:
from transformers import TFBertModel, TFAutoModel
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
from transformers import BertTokenizer, AutoTokenizer

# Initialize the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize and convert text to input features with attention mask
max_seq_length = 128

def preprocess_text(text):
    tokens = tokenizer(text, max_length=max_seq_length, truncation=True, padding="max_length", return_attention_mask=True)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    return input_ids, attention_mask

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
bert_X_train = pd.DataFrame()
bert_X_test = pd.DataFrame()
bert_X_val = pd.DataFrame()

In [ ]:
#df[['input_ids', 'attention_mask']] = df['text'].apply(preprocess_text).apply(pd.Series)
bert_X_train[['input_ids', 'attention_mask']] = X_train['text'].apply(preprocess_text).apply(pd.Series)
bert_X_test[['input_ids', 'attention_mask']]= X_test['text'].apply(preprocess_text).apply(pd.Series)
bert_X_val[['input_ids', 'attention_mask']] = X_val['text'].apply(preprocess_text).apply(pd.Series)

In [ ]:
X_train_ids = np.array(bert_X_train['input_ids'].tolist())
X_train_mask = np.array(bert_X_train['attention_mask'].tolist())

X_test_ids = np.array(bert_X_test['input_ids'].tolist())
X_test_mask = np.array(bert_X_test['attention_mask'].tolist())

X_val_ids = np.array(bert_X_val['input_ids'].tolist())
X_val_mask = np.array(bert_X_val['attention_mask'].tolist())

In [ ]:
X_train_ids

array([[  101,  1051, 11923, ...,  2175,  2361,   102],
       [  101, 20312, 26665, ...,  3038, 12478,   102],
       [  101,  6015,  2331, ...,  4071,  4613,   102],
       ...,
       [  101,  3994, 26665, ...,     0,     0,     0],
       [  101,  5977, 17688, ...,  2812,  4078,   102],
       [  101, 13857,  8112, ...,  4895,  3432,   102]])

In [ ]:
X_train_mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

# BERT Model without length

In [ ]:
input_layer = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='attention_mask')
bert_output = bert_model([input_layer, attention_mask])[0]

In [ ]:
from tensorflow.keras.optimizers import Adam

# Flatten or reshape the BERT output
flatten_layer = Flatten()(bert_output)

# Define DNN layers
dense_layer_1 = Dense(128, activation='relu')(flatten_layer)
dropout_layer_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_2)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)

## without finetune learning rate

In [ ]:
# Define the model
model = tf.keras.models.Model(inputs=[input_layer, attention_mask], outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 5

history = model.fit(
    x={'input_ids': X_train_ids, 'attention_mask': X_train_mask},
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)
)

Epoch 1/5
941/941 [==============================] - 755s 803ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481
Epoch 2/5
941/941 [==============================] - 752s 799ms/step - loss: 0.6904 - accuracy: 0.5382 - val_loss: 0.6887 - val_accuracy: 0.5481
Epoch 3/5
941/941 [==============================] - 752s 799ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6886 - val_accuracy: 0.5481
Epoch 4/5
941/941 [==============================] - 752s 799ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6886 - val_accuracy: 0.5481
Epoch 5/5
941/941 [==============================] - 752s 799ms/step - loss: 0.6902 - accuracy: 0.5382 - val_loss: 0.6889 - val_accuracy: 0.5481


## with fine-tune learning rate

In [ ]:
# Define the model
bert_finetuned_model= tf.keras.models.Model(inputs=[input_layer, attention_mask], outputs=output_layer)
bert_finetuned_model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 5

history = bert_finetuned_model.fit(
    x={'input_ids': X_train_ids, 'attention_mask': X_train_mask},
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=({'input_ids': X_val_ids, 'attention_mask': X_val_mask}, y_val)
)

Epoch 1/5


941/941 [==============================] - 788s 801ms/step - loss: 0.6902 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481
Epoch 2/5
941/941 [==============================] - 751s 798ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481
Epoch 3/5
941/941 [==============================] - 751s 798ms/step - loss: 0.6902 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481
Epoch 4/5
941/941 [==============================] - 750s 797ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481
Epoch 5/5
941/941 [==============================] - 751s 798ms/step - loss: 0.6903 - accuracy: 0.5382 - val_loss: 0.6888 - val_accuracy: 0.5481


# BERT Model with length

In [ ]:
bert_X_train_ids = pd.Series(X_train_ids.tolist())
bert_X_test_ids = pd.Series(X_test_ids.tolist())
bert_X_val_ids = pd.Series(X_val_ids.tolist())

bert_X_train_mask = pd.Series(X_train_mask.tolist())
bert_X_test_mask = pd.Series(X_test_mask.tolist())
bert_X_val_mask = pd.Series(X_val_mask.tolist())

In [ ]:
min_values_bert = min(bert_X_train_ids.apply(lambda x: min(x)))
max_values_bert = max(bert_X_train_ids.apply(lambda x: max(x)))

In [ ]:
# Normalize to the range (a, b)
normalized_X_train_ids_bert = min_values_bert + (X_train['length_info'] - X_train['length_info'].min()) * (max_values_bert - min_values_bert) / (X_train['length_info'].max() - X_train['length_info'].min())
normalized_X_test_ids_bert = min_values_bert + (X_test['length_info'] - X_test['length_info'].min()) * (max_values_bert - min_values_bert) / (X_test['length_info'].max() - X_test['length_info'].min())
normalized_X_val_ids_bert = min_values_bert + (X_val['length_info'] - X_val['length_info'].min()) * (max_values_bert - min_values_bert) / (X_val['length_info'].max() - X_val['length_info'].min())

In [ ]:
bert_X_train_ids_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(bert_X_train_ids, normalized_X_train_ids_bert)])
bert_X_test_ids_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(bert_X_test_ids, normalized_X_test_ids_bert)])
bert_X_val_ids_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(bert_X_val_ids, normalized_X_val_ids_bert)])

In [ ]:
bert_X_train_ids_withlen = np.array(bert_X_train_ids_withlen.tolist())
bert_X_test_ids_withlen = np.array(bert_X_test_ids_withlen.tolist())
bert_X_val_ids_withlen = np.array(bert_X_val_ids_withlen.tolist())

In [ ]:
array_1d = pd.Series(np.ones(len(X_train_mask)))

In [ ]:
bert_X_train_mask_withlen = np.array([arr + [length_info_item] for arr, length_info_item in zip(bert_X_train_mask, array_1d)])
bert_X_test_mask_withlen = np.array([arr + [length_info_item] for arr, length_info_item in zip(bert_X_test_mask, array_1d)])
bert_X_val_mask_withlen = np.array([arr + [length_info_item] for arr, length_info_item in zip(bert_X_val_mask, array_1d)])

In [ ]:
input_layer_withlen = tf.keras.layers.Input(shape=(max_seq_length+1,), dtype=tf.int32, name='input_ids_withlen')
attention_mask_withlen = tf.keras.layers.Input(shape=(max_seq_length+1,), dtype=tf.int32, name='attention_mask_withlen')
bert_output_withlen = bert_model([input_layer_withlen, attention_mask_withlen])[0]

In [ ]:
# Flatten or reshape the BERT output
flatten_layer = Flatten()(bert_output_withlen)

# Define your DNN layers
dense_layer_1 = Dense(128, activation='relu')(flatten_layer)
dropout_layer_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_2)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)

# Define the model
bert_model_withlen = tf.keras.models.Model(inputs=[input_layer_withlen, attention_mask_withlen], outputs=output_layer)
bert_model_withlen.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

#bert_model_withlen.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10

history = bert_model_withlen.fit(
    x={'input_ids_withlen': bert_X_train_ids_withlen, 'attention_mask_withlen': bert_X_train_mask_withlen},
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=({'input_ids_withlen': bert_X_val_ids_withlen, 'attention_mask_withlen': bert_X_val_mask_withlen}, y_val)
)

Epoch 1/10


941/941 [==============================] - 855s 872ms/step - loss: 0.6969 - accuracy: 0.5261 - val_loss: 0.6928 - val_accuracy: 0.5481
Epoch 2/10
941/941 [==============================] - 775s 823ms/step - loss: 0.6926 - accuracy: 0.5380 - val_loss: 0.6920 - val_accuracy: 0.5481
Epoch 3/10
941/941 [==============================] - 772s 820ms/step - loss: 0.6921 - accuracy: 0.5374 - val_loss: 0.6914 - val_accuracy: 0.5481
Epoch 4/10
941/941 [==============================] - 770s 819ms/step - loss: 0.6916 - accuracy: 0.5382 - val_loss: 0.6909 - val_accuracy: 0.5481
Epoch 5/10
941/941 [==============================] - 771s 819ms/step - loss: 0.6913 - accuracy: 0.5382 - val_loss: 0.6905 - val_accuracy: 0.5481
Epoch 6/10
941/941 [==============================] - 775s 823ms/step - loss: 0.6912 - accuracy: 0.5377 - val_loss: 0.6903 - val_accuracy: 0.5481
Epoch 7/10
941/941 [==============================] - 772s 820ms/step - loss: 0.6910 - accuracy: 0.5378 - val_loss: 0.6900 - val_accura

# BERT - Active Learning

In [ ]:
import random

In [ ]:
input_layer_withlen = tf.keras.layers.Input(shape=(max_seq_length+1,), dtype=tf.int32, name='input_ids_withlen')
attention_mask_withlen = tf.keras.layers.Input(shape=(max_seq_length+1,), dtype=tf.int32, name='attention_mask_withlen')
bert_output_withlen = bert_model([input_layer_withlen, attention_mask_withlen])[0]

In [ ]:
# Flatten or reshape the BERT output
flatten_layer = Flatten()(bert_output_withlen)

# DNN layers
dense_layer_1 = Dense(128, activation='relu')(flatten_layer)
dropout_layer_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_2)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)

In [ ]:
# Active learning loop
query_indices = random.sample(range(len(bert_X_train_ids_withlen)), k=100)  # Initialize with a random set of instances

In [ ]:
# Set the number of active learning iterations
num_iterations = 10

for i in range(num_iterations):
    # Define the model
    bert_model_active = tf.keras.models.Model(inputs=[input_layer_withlen, attention_mask_withlen], outputs=output_layer)
    #bert_model_active.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    bert_model_active.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model on the current training set
    bert_model_active.fit( x={'input_ids_withlen': bert_X_train_ids_withlen[query_indices], 'attention_mask_withlen': bert_X_train_mask_withlen[query_indices]}, y=y_train.iloc[query_indices], epochs=5, batch_size=32, verbose=1)

    # Predict on the test set
    test_predictions = (bert_model_active.predict(x={'input_ids_withlen': bert_X_val_ids_withlen, 'attention_mask_withlen': bert_X_val_mask_withlen}) > 0.5).astype(int)

    # Evaluate the model
    accuracy = accuracy_score(y_val, test_predictions)
    print(f"Iteration {i + 1} - Validation Accuracy: {accuracy}")

    # Query the instances with the least confidence
    uncertainty_scores = np.abs(bert_model_active.predict(x ={'input_ids_withlen': bert_X_train_ids_withlen, 'attention_mask_withlen': bert_X_train_mask_withlen}) - 0.5).flatten()
    #query_indices = np.argsort(uncertainty_scores)[-100:]

    # Add the queried instances to the training set
    query_indices = np.concatenate((query_indices, np.argsort(uncertainty_scores)[-100:]))

Epoch 1/5


4/4 [==============================] - 39s 246ms/step - loss: 2.9300 - accuracy: 0.5400
Epoch 2/5
4/4 [==============================] - 1s 244ms/step - loss: 42.8057 - accuracy: 0.4400
Epoch 3/5
4/4 [==============================] - 1s 208ms/step - loss: 10.3560 - accuracy: 0.4600
Epoch 4/5
4/4 [==============================] - 1s 235ms/step - loss: 5.9048 - accuracy: 0.4700
Epoch 5/5
202/202 [==============================] - 23s 98ms/step
Iteration 1 - Validation Accuracy: 0.548062015503876
941/941 [==============================] - 93s 99ms/step
Epoch 1/5


7/7 [==============================] - 39s 263ms/step - loss: 7.0923 - accuracy: 0.4150
Epoch 2/5
7/7 [==============================] - 2s 259ms/step - loss: 3.4176 - accuracy: 0.5850
Epoch 3/5
7/7 [==============================] - 2s 262ms/step - loss: 1.7447 - accuracy: 0.5900
Epoch 4/5
7/7 [==============================] - 2s 264ms/step - loss: 0.7116 - accuracy: 0.6500
Epoch 5/5
202/202 [==============================] - 25s 99ms/step
Iteration 2 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 93s 98ms/step
Epoch 1/5


10/10 [==============================] - 38s 269ms/step - loss: 0.6777 - accuracy: 0.6700
Epoch 2/5
10/10 [==============================] - 3s 267ms/step - loss: 0.6721 - accuracy: 0.6700
Epoch 3/5
10/10 [==============================] - 3s 271ms/step - loss: 0.6659 - accuracy: 0.6700
Epoch 4/5
10/10 [==============================] - 3s 265ms/step - loss: 0.6615 - accuracy: 0.6700
Epoch 5/5
202/202 [==============================] - 22s 98ms/step
Iteration 3 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 93s 98ms/step
Epoch 1/5


13/13 [==============================] - 42s 274ms/step - loss: 0.6625 - accuracy: 0.6375
Epoch 2/5
13/13 [==============================] - 4s 272ms/step - loss: 0.6605 - accuracy: 0.6375
Epoch 3/5
13/13 [==============================] - 4s 270ms/step - loss: 0.6571 - accuracy: 0.6375
Epoch 4/5
13/13 [==============================] - 4s 271ms/step - loss: 0.6564 - accuracy: 0.6375
Epoch 5/5
202/202 [==============================] - 23s 98ms/step
Iteration 4 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 93s 98ms/step
Epoch 1/5


...

19/19 [==============================] - 43s 276ms/step - loss: 0.6718 - accuracy: 0.6050
Epoch 2/5
19/19 [==============================] - 5s 277ms/step - loss: 0.6716 - accuracy: 0.6050
Epoch 3/5
19/19 [==============================] - 5s 277ms/step - loss: 0.6704 - accuracy: 0.6050
Epoch 4/5
19/19 [==============================] - 5s 276ms/step - loss: 0.6707 - accuracy: 0.6050
Epoch 5/5
202/202 [==============================] - 22s 98ms/step
Iteration 6 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 92s 98ms/step
Epoch 1/5


22/22 [==============================] - 44s 280ms/step - loss: 0.6762 - accuracy: 0.5957
Epoch 2/5
22/22 [==============================] - 6s 278ms/step - loss: 0.6740 - accuracy: 0.5957
Epoch 3/5
22/22 [==============================] - 6s 277ms/step - loss: 0.6741 - accuracy: 0.5957
Epoch 4/5
22/22 [==============================] - 6s 276ms/step - loss: 0.6765 - accuracy: 0.5957
Epoch 5/5
202/202 [==============================] - 23s 98ms/step
Iteration 7 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 92s 98ms/step
Epoch 1/5


25/25 [==============================] - 38s 263ms/step - loss: 0.6796 - accuracy: 0.5888
Epoch 2/5
25/25 [==============================] - 7s 262ms/step - loss: 0.6779 - accuracy: 0.5888
Epoch 3/5
25/25 [==============================] - 7s 262ms/step - loss: 0.6777 - accuracy: 0.5888
Epoch 4/5
25/25 [==============================] - 7s 262ms/step - loss: 0.6766 - accuracy: 0.5888
Epoch 5/5
202/202 [==============================] - 22s 98ms/step
Iteration 8 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 92s 98ms/step
Epoch 1/5


29/29 [==============================] - 45s 276ms/step - loss: 0.6785 - accuracy: 0.5833
Epoch 2/5
29/29 [==============================] - 8s 273ms/step - loss: 0.6797 - accuracy: 0.5833
Epoch 3/5
29/29 [==============================] - 8s 273ms/step - loss: 0.6804 - accuracy: 0.5833
Epoch 4/5
29/29 [==============================] - 8s 275ms/step - loss: 0.6797 - accuracy: 0.5833
Epoch 5/5
202/202 [==============================] - 22s 98ms/step
Iteration 9 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 92s 98ms/step
Epoch 1/5


32/32 [==============================] - 47s 276ms/step - loss: 0.6812 - accuracy: 0.5790
Epoch 2/5
32/32 [==============================] - 9s 275ms/step - loss: 0.6804 - accuracy: 0.5790
Epoch 3/5
32/32 [==============================] - 9s 274ms/step - loss: 0.6803 - accuracy: 0.5790
Epoch 4/5
32/32 [==============================] - 9s 275ms/step - loss: 0.6799 - accuracy: 0.5790
Epoch 5/5
202/202 [==============================] - 23s 98ms/step
Iteration 10 - Validation Accuracy: 0.45193798449612405
941/941 [==============================] - 92s 98ms/step


# 5. FastText embeddings

In [ ]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 829.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=92d77e3ce4f09cdd5d4eccded2ab229a1dd447622fa5f2eb4501110ae2447ba8
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext
from keras.layers import Dense, Embedding, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))  # Convert to a list of floats
    return data

In [ ]:
fasttext_model = load_vectors(directory + 'wiki-news-300d-1M.vec')

In [ ]:
# Load your dataset (replace 'fake_news.csv' and column names with your dataset)
X = df['text'].values
#y = df['target'].values

# Preprocess your text data
max_seq_length = 512  # Define a maximum sequence length
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
#X = tokenizer.texts_to_sequences(X)
#X = pad_sequences(X, maxlen=max_seq_length)

X_train_fasttext = tokenizer.texts_to_sequences(X_train['text'])
X_train_fasttext = pad_sequences(X_train_fasttext, maxlen=max_seq_length)

X_test_fasttext = tokenizer.texts_to_sequences(X_test['text'])
X_test_fasttext = pad_sequences(X_test_fasttext, maxlen=max_seq_length)

X_val_fasttext = tokenizer.texts_to_sequences(X_val['text'])
X_val_fasttext = pad_sequences(X_val_fasttext, maxlen=max_seq_length)

In [ ]:
# Create a matrix for the FastText word vectors
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        embedding_matrix[i] = fasttext_model[word]

In [ ]:
# Function to convert tokenized and padded text to FastText embeddings using embedding_matrix
def text_to_embeddings(tokenized_text, embedding_matrix):
    word_vectors = []

    for token_id in tokenized_text:
        # Map token_id to its corresponding FastText embedding using the embedding_matrix
        # Note that token_id should correspond to a word in the vocabulary
        if token_id < embedding_matrix.shape[0]:  # Ensure token_id is within the vocabulary range
            word_vectors.append(embedding_matrix[token_id])
        else:
            # Handle out-of-vocabulary token_ids (e.g., use zeros)
            word_vectors.append(np.zeros(embedding_matrix.shape[1]))  # Zero vector of the same dimension

    # Calculate the average vector for the entire text
    if word_vectors:
        avg_vector = np.mean(word_vectors, axis=0)
    else:
        avg_vector = np.zeros(embedding_matrix.shape[1])  # Zero vector of the same dimension

    return avg_vector

In [ ]:
# Convert tokenized and padded text data in X to FastText embeddings
#X_fasttext = np.array([text_to_embeddings(tokenized_text, embedding_matrix) for tokenized_text in X])

fasttext_X_train = np.array([text_to_embeddings(tokenized_text, embedding_matrix) for tokenized_text in X_train_fasttext])
fasttext_X_test = np.array([text_to_embeddings(tokenized_text, embedding_matrix) for tokenized_text in X_test_fasttext])
fasttext_X_val = np.array([text_to_embeddings(tokenized_text, embedding_matrix) for tokenized_text in X_val_fasttext])

In [ ]:
fasttext_X_train

array([[-1.37089844e-03, -8.12890625e-04, -1.55267578e-02, ...,
         3.41292969e-02,  1.01056641e-02, -3.67617188e-03],
       [ 1.06121094e-02, -5.02402344e-03,  4.43671875e-03, ...,
         2.76525391e-02,  2.11894531e-03, -1.15917969e-03],
       [ 7.18339844e-03, -4.78398437e-03, -5.26757812e-04, ...,
         7.42353516e-02,  6.97148437e-03, -1.79335937e-03],
       ...,
       [ 5.29355469e-03,  1.11210937e-03,  7.13085938e-04, ...,
         2.08904297e-02,  1.37519531e-03,  6.55273437e-04],
       [-1.79433594e-03, -9.07792969e-03, -3.82363281e-03, ...,
         1.02952930e-01,  1.54152344e-02, -1.02156250e-02],
       [ 1.20419922e-02,  4.29687500e-06,  1.39777344e-02, ...,
         6.65011719e-02,  8.84687500e-03, -1.84472656e-03]])

# FASTTEXT Model without length

In [ ]:
fasttext_X_train.shape

(30098, 300)

In [ ]:
fast_model = create_dnn_model(fasttext_X_train)
fast_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fast_model.fit(fasttext_X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 9s 3ms/step - loss: 0.3411 - accuracy: 0.8637
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2012 - accuracy: 0.9274
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1816 - accuracy: 0.9354
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1669 - accuracy: 0.9395
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1578 - accuracy: 0.9441
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1559 - accuracy: 0.9439
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1523 - accuracy: 0.9451
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1453 - accuracy: 0.9467
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1405 - accuracy: 0.9505
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1398 - accuracy: 0.9487

In [ ]:
# Evaluate the model
loss, accuracy = fast_model.evaluate(fasttext_X_val, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1358 - accuracy: 0.9512
Test loss: 0.13582593202590942
Test accuracy: 0.9511628150939941


# FASTTEXT Model with length

In [ ]:
fasttext_X_train_list = pd.Series(fasttext_X_train.tolist())
fasttext_X_test_list = pd.Series(fasttext_X_test.tolist())
fasttext_X_val_list = pd.Series(fasttext_X_val.tolist())

In [ ]:
min_values_fast = min(fasttext_X_train_list.apply(lambda x: min(x)))
max_values_fast= max(fasttext_X_train_list.apply(lambda x: max(x)))

In [ ]:
# Normalize to the range (a, b)
normalized_data_X_train_fast = min_values_fast + (X_train['length_info'] - X_train['length_info'].min()) * (max_values_fast - min_values_fast) / (X_train['length_info'].max() - X_train['length_info'].min())
normalized_data_X_test_fast = min_values_fast + (X_test['length_info'] - X_test['length_info'].min()) * (max_values_fast - min_values_fast) / (X_test['length_info'].max() - X_test['length_info'].min())
normalized_data_X_val_fast = min_values_fast + (X_val['length_info'] - X_val['length_info'].min()) * (max_values_fast - min_values_fast) / (X_val['length_info'].max() - X_val['length_info'].min())

In [ ]:
fast_X_train_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(fasttext_X_train_list, normalized_data_X_train_fast)])
fast_X_test_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(fasttext_X_test_list, normalized_data_X_test_fast)])
fast_X_val_withlen = pd.Series([arr + [length_info_item] for arr, length_info_item in zip(fasttext_X_val_list, normalized_data_X_val_fast)])

In [ ]:
fast_X_train_withlen = np.array(fast_X_train_withlen.tolist())
fast_X_test_withlen = np.array(fast_X_test_withlen.tolist())
fast_X_val_withlen = np.array(fast_X_val_withlen.tolist())

In [ ]:
fast_model_withlen = create_dnn_model(fast_X_train_withlen)
fast_model_withlen.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fast_model_withlen.fit(fast_X_train_withlen, y_train, epochs=10, batch_size=32)

Epoch 1/10
941/941 [==============================] - 4s 3ms/step - loss: 0.3474 - accuracy: 0.8524
Epoch 2/10
941/941 [==============================] - 3s 3ms/step - loss: 0.2049 - accuracy: 0.9254
Epoch 3/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1845 - accuracy: 0.9339
Epoch 4/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1698 - accuracy: 0.9389
Epoch 5/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1633 - accuracy: 0.9402
Epoch 6/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1586 - accuracy: 0.9426
Epoch 7/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1529 - accuracy: 0.9449
Epoch 8/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1486 - accuracy: 0.9463
Epoch 9/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1470 - accuracy: 0.9468
Epoch 10/10
941/941 [==============================] - 3s 3ms/step - loss: 0.1412 - accuracy: 0.9498

In [ ]:
# Evaluate the model
loss, accuracy = fast_model_withlen.evaluate(fast_X_val_withlen, y_val)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

202/202 [==============================] - 1s 2ms/step - loss: 0.1474 - accuracy: 0.9464
Test loss: 0.14737294614315033
Test accuracy: 0.9463565945625305


# Fasttext - Active Learning

In [ ]:
import random

In [ ]:
# Active learning loop
query_indices = random.sample(range(len(fast_X_train_withlen)), k=100)  # Initialize with a random set of instances

In [ ]:
# Set the number of active learning iterations
num_iterations = 10

for i in range(num_iterations):
    model_fast_withlen_active = create_dnn_model(fast_X_train_withlen)
    model_fast_withlen_active.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model on the current training set
    model_fast_withlen_active.fit(fast_X_train_withlen[query_indices], y_train.iloc[query_indices], epochs=10, batch_size=32, verbose=1)

    # Predict on the test set
    test_predictions = (model_fast_withlen_active.predict(fast_X_val_withlen) > 0.5).astype(int)

    # Evaluate the model
    accuracy = accuracy_score(y_val, test_predictions)
    print(f"Iteration {i + 1} - Validation Accuracy: {accuracy}")

    # Query the instances with the least confidence
    uncertainty_scores = np.abs(model_fast_withlen_active.predict(fast_X_train_withlen) - 0.5).flatten()
    #query_indices = np.argsort(uncertainty_scores)[-100:]

    # Add the queried instances to the training set
    query_indices = np.concatenate((query_indices, np.argsort(uncertainty_scores)[-100:]))

Epoch 1/10
4/4 [==============================] - 1s 5ms/step - loss: 0.7001 - accuracy: 0.4800
Epoch 2/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6957 - accuracy: 0.5200
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6944 - accuracy: 0.5200
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6958 - accuracy: 0.4200
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5300
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6834 - accuracy: 0.6200
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6847 - accuracy: 0.5700
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 9/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6844 - accuracy: 0.5400
Epoch 10/10
202/202 [==============================] - 0s 1ms/step
Iteration 1 - Validation Accuracy: 0.5444961240310078
941/941 [======

In [ ]:
# Final evaluation on the test set
final_test_predictions = (model_fast_withlen_active.predict(fast_X_val_withlen) > 0.5).astype(int)
final_accuracy = accuracy_score(y_val, final_test_predictions)
print(f"Final Validation Accuracy: {final_accuracy}")


202/202 [==============================] - 0s 1ms/step
Final Validation Accuracy: 0.6776744186046512
